In [7]:
import fiona
from shapely.geometry import Point, Polygon
import geopandas as gpd
import pandas as pd

import numpy as np
import pandas as pd
import datetime
from datetime import datetime
import os
import datetime
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import math

from w210_attribute_library_scale import shAttributes, getdate, haversine_distance, tilekey, date_string

print("Date of reference: ", date_string)

datdir = '../data/'
attrs = "../attrs/"

Date of reference:  2022/06/01


## Read Sinkhole Data

In [8]:
df_sh = pd.read_excel(datdir+"Florida_Subsidence_Incident_Reports.xlsx", sheet_name=0)
df_sh["DateD"] = df_sh.apply(lambda row: getdate(row["EVENT_DATE"]), axis=1)
df_sh = df_sh[~df_sh["EVENT_DATE"].isna()]
print(len(df_sh))
df_sh.head(2)

3944


/var/folders/f3/z2y9_l5x2ddfrpzb4m7jw6_w0000gn/T/ipykernel_21207/3280829123.py:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df_sh["DateD"] = df_sh.apply(lambda row: getdate(row["EVENT_DATE"]), axis=1)


,X,Y,OBJECTID,REF_NUM,DATE_REV,EVENT_DATE,Year,Month,TRUE_SINK,LONGDD,...,CAVVIS,SUBRATE,PROPDAM,REPAIR_S,DRAINSTR,SOILTYPE,COMMENTS,COMMENTS_2,ACCESS_,DateD
0,-82.187100,27.758900,1,10-769,2015/12/23 00:00:00+00,1974/04/07 00:00:00+00,1974.0,4.0,U,-82.187097,...,U,U,Y,Unknown,Unknown,UNKNOWN,BALM SINKHOLE. FIVE ORANGE TREES FELL INTO HOL...,"LEFT IN POSITION, AS GIVEN BY CK 10/5/12. ONE ...",9.1 MILES EAST OF S.R. 301 ON S.R. 672. 1350 F...,1974-04-07
1,-81.799854,28.667132,2,11-538,2016/04/28 00:00:00+00,1989/12/24 00:00:00+00,1989.0,12.0,U,-81.799850,...,U,R,N,Planned,Unknown,UNK,THERE IS TWO SINKHOLES ON THIS SITE; SMALL SIN...,Accurate to the groves in section 10,"HI-ACRE GROVES, O'BRIEN RD. CLERMONT, FL ...",1989-12-24


## Read Tile Data

In [9]:
ftileslarge = 'scale_up_F_satellite_ws_soil_karst_attr.csv'
dftiles_large = pd.read_csv(attrs+ftileslarge)
# dftiles_large["Key"] = dftiles_large.apply(lambda row: tilekey(row), axis=1)
dftiles_large["DateD"] = datetime.strptime(date_string,"%Y/%m/%d")
print(len(dftiles_large))
print(len(dftiles_large["Key"].unique()))
dftiles_large.head(2)

407463
407463


,Key,Calcium Carbonate,Gypsum,Soil Health Organic Matter,Percent Clay,Percent Sand,Percent Silt,Available Water Storage,date_ws,name_x,...,Industrial,Pasture,PermanentCrop,Residential,River,SeaLake,prediction,prediction_name,gridcode,DateD
0,033_174,0.0,0,6.177,10.1146,57.0835,8.9811,18.52059,2021-06-01,WHITING FIELD NAVAL AIR STATI,...,1.737650e-16,1.376721e-14,3.401483e-10,6.664154e-13,0.054241,0.933093,9,SeaLake,1,2022-06-01
1,033_206,0.0,0,6.177,10.1146,57.0835,8.9811,18.52059,2021-06-01,WHITING FIELD NAVAL AIR STATI,...,2.333318e-17,3.809984e-13,4.369865e-11,7.213560e-14,0.094416,0.016788,1,Forest,1,2022-06-01


In [10]:
dftiles_large.columns

Index(['Key', 'Calcium Carbonate', 'Gypsum', 'Soil Health Organic Matter',
       'Percent Clay', 'Percent Sand', 'Percent Silt',
       'Available Water Storage', 'date_ws', 'name_x', 'lon_t', 'lat_t',
       'rolling_7_precip', 'rolling_15_precip', 'rolling_30_precip',
       'rolling_60_precip', 'rolling_90_precip', 'y1_mean_prc', 'y1_max_prc',
       'y1_mean_tmp', 'y1_max_tmp', 'y1_min_tmp', 'y2_mean_prc', 'y2_max_prc',
       'y2_mean_tmp', 'y2_max_tmp', 'y2_min_tmp', 'y3_mean_prc', 'y3_max_prc',
       'y3_mean_tmp', 'y3_max_tmp', 'y3_min_tmp', 'County', 'county_fp',
       'name_y', 'imagenum', 'geometry', 'AnnualCrop', 'Forest',
       'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture',
       'PermanentCrop', 'Residential', 'River', 'SeaLake', 'prediction',
       'prediction_name', 'gridcode', 'DateD'],
      dtype='object')

## Generating Sinkhole Features

In [11]:
daysdelta = timedelta(0)
fields = ["lon_t", "lat_t", "Key"]
large_tiles_sh = shAttributes(df_sh, dftiles_large, daysdelta, fields)
print(len(large_tiles_sh))
large_tiles_sh.head(2)

26
407463


,Key,Calcium Carbonate,Gypsum,Soil Health Organic Matter,Percent Clay,Percent Sand,Percent Silt,Available Water Storage,date_ws,name_x,...,Y100,Y150,Y200,Y250,Y300,Y500,Y750,Y1000,Y1000plus,Ycoloc
0,033_174,0.0,0,6.177,10.1146,57.0835,8.9811,18.52059,2021-06-01,WHITING FIELD NAVAL AIR STATI,...,0,0,0,0,0,0,0,0,101533,0
1,033_206,0.0,0,6.177,10.1146,57.0835,8.9811,18.52059,2021-06-01,WHITING FIELD NAVAL AIR STATI,...,0,0,0,0,0,0,0,0,101533,0


In [12]:
large_tiles_sh.to_csv(attrs+"scale_up_F_satellite_ws_soil_karst_sh_attr_F.csv")